めも

- 全部のデータをくっつける
- arモデルに突っ込む
- mlflowを使用する
- bootstrap信頼区間を調べる

In [ ]:
import sys
sys.version

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns 
import statsmodels.tsa.api as ts
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# 評価
$$
RMSLE = \sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2}
$$

# データの確認

- エクアドルは石油依存国なので、石油価格に経済が影響されやすい
- 公務員の給料は15日と月末に支払われる
- 2016年4月16日にマグニチュード 7.8 の地震がエクアドルを襲う

- 休日について（翻訳ﾏﾏ）
    - 休日とイベント、メタデータ付き
    - 注:転送された列に特に注意してください。正式に移管された祝日は、その暦日に該当しますが、政府によって別の日付に移動されました。振替日は、休日というより通常の日に近いものです。実際に祝われた日を見つけるには、タイプが Transfer である対応する行を探します。たとえば、祝日の Independencia de Guayaquil は 2012-10-09 から 2012-10-12 に変更されました。つまり、2012-10-12 に祝われました。ブリッジ タイプの日は、休日に追加される余分な日です (たとえば、長い週末に休憩を延長するため)。これらは、Bridge の返済を目的とした、通常は予定されていない勤務日 (例: 土曜日) である Work Day のタイプによって構成されていることがよくあります。
    - 追加の休日とは、通常のカレンダーの休日に追加される日です。たとえば、通常はクリスマスの前後に発生します (クリスマス イブを休日にします)。
    
- 単語の共有
    - store_nbr→店舗ID
    - family→製品カテゴリ

## 要約

- 欠損値なし
- 2013/01/01 ～ 2017/08/15までが学習データで、2017/08/16 ～ 2017/08/31 までを予測
- 店舗は全部で54店舗あり、それぞれの店舗は33の製品カテゴリを持っている

## データの読み込み

In [ ]:
train = pd.read_csv("data/train.csv", parse_dates=["date"])
test = pd.read_csv("data/test.csv", parse_dates=["date"])
holidays_events = pd.read_csv("data/holidays_events.csv", parse_dates=["date"])
oil = pd.read_csv("data/oil.csv", parse_dates=["date"])
store = pd.read_csv("data/stores.csv")
transactions = pd.read_csv("data/transactions.csv", parse_dates=["date"])

train["is_train"] = True
test["is_train"] = False

df = pd.concat([train, test], ignore_index=True, axis=0)

- 2013/01/01 ～ 2017/08/15までが学習データ

In [ ]:
train["date"].unique()

- 2017/08/16 ～ 2017/08/31 までを予測
- 単純なarだと無理そう

In [ ]:
test["date"].unique()

- 店舗は全部で54
- 店舗全体の売上の25%が0である
- 製品カテゴリは全部で33ある

In [ ]:
train.shape, test.shape, df["store_nbr"].unique().shape

### 学習データ

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.groupby("family").size()

### テストデータ

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
test.groupby("family").size()

### メタデータ

In [ ]:
holidays_events.head()

In [ ]:
oil.head()

In [ ]:
store.head()

In [ ]:
store.groupby("city").size()

In [ ]:
store.groupby("state").size()

In [ ]:
store.groupby("type").size()

In [ ]:
store.groupby("cluster").size()

In [ ]:
transactions.head()

## 欠損値チェック

- 欠損なし

In [ ]:
train.isnull().sum().sort_values(ascending=False)

In [ ]:
test.isnull().sum().sort_values(ascending=False)

## 可視化

### オイル価格について

- 2015年ごろに大幅に原油価格が下落
- （1バレル当たり）2014年6月20日：107・95ドル → 2015年12月28日：36・36ドル。実に66％の下落
- 原因はアメリカのシューオイルを輸出するための法改正が行われたため　[詳しく](https://diamond.jp/articles/-/88442)

In [ ]:
sns.lineplot(
    x="date",
    y="dcoilwtico",
    data=oil,
)
plt.title("oil price")

- 

- 偏自己相関
    - 偏自己相関はラグ2以降は有意な値になっていないため、ARモデルで表すとAR(1)がよさそう
- 自己相関
    - 自己相関は減衰傾向にあるためARモデルとMAモデルを比べるとARモデルのほうがよく表せそう

Box–Jenkins method

In [ ]:
plot_pacf(oil["dcoilwtico"].dropna(), method="ywm");

In [ ]:
plot_acf(oil["dcoilwtico"], missing="drop");

### 製品カテゴリの売上について

- 売上が0の製品カテゴリが存在している

In [ ]:
fig, axes = plt.subplots(len(familys), 1, figsize=(20, 100))
for i, family in enumerate(familys):
    df = train.query("store_nbr==1").query("family=='{}'".format(family))
    axes[i].plot(df["date"], df["sales"])
    axes[i].set_title(family)

- 同一クラスターの店舗の売上を見てみる
    - store_nbr=1はクラスター13に属している
    - クラスター13に属しているstore_nbr={1, 2, 6, 53}
- おおむね同じような概形をしているので、クラスター毎に学習させるのがいいかもしれない

In [ ]:
familys = train["family"].unique()
fig, axes = plt.subplots(len(familys), 2, figsize=(20, 120))
for j, id in enumerate([1, 2]):
    for i, family in enumerate(familys):
        df = train.query("store_nbr==@id").query("family=='{}'".format(family))
        axes[i, j].plot(df["date"], df["sales"])
        axes[i, j].set_title(family + "store_nbr={}".format(id))